# API Nakala

In [ ]:
# ANR experts
# author: @sardinecan
# date: 2022-12
# description: this Julia notebook allows to interact with the Nakala's API
# licence: CC-0

# @todo : gestion des erreurs (réponses server) ?
# @todo : écrire un fichier de log pour récupérer les identifiants des ressources crées.

## Packages

In [ ]:
path = @__DIR__
cd(path)

In [ ]:
using Pkg
Pkg.instantiate() 
Pkg.activate(path)
using CSV
using DataFrames
using HTTP
using JSON
using Dates
using FileIO
using Images

## Identifiants

In [ ]:
path = @__DIR__
credentials = CSV.read(joinpath(path, "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [ ]:
apitest = true

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

## Librairie de fonctions

In [ ]:
newARGS = apiKey
include("functions.jl")

## Collections

### Création d'une collection

Une fois les identifiants chargés et l'API sélectionnée, entrez simplement un nom de collection pour la créer.

In [ ]:
collectionName = "testCollection"
createCollection(collectionName)

### Ajouter des données à une collection

In [ ]:
collectionIdentifier = "10.34847/nkl.5c519k06"
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

addDatas2Collection(collectionIdentifier, datas)

### Supprimer des données d'une collection

In [ ]:
collectionIdentifier = ""
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

removeDatasFromCollection(collectionIdentifier, datas)

## Utilisateurs

### Informations utilisateur

In [ ]:
userInfo()

### Ressources utilisateur

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
getUserDatas(scope[1])

### Lister les données privées/publiques

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ] #[ "pending", "published" ]

getUserDatas(scope[1], status[2])

### Publier les données privées

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(getUserDatas(scope[1], status[1]), "data", "")
pendingDatas = [get(data, "identifier", "") for data in datas]

[publishData(pendingData) for pendingData in pendingDatas]

### Lister les données

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(getUserDatas(scope[1]), "data", "nothing")

list = Vector()
for data in datas
  
  identifier = get(data, "identifier", "")
  metas = get(data, "metas", "")
  #push!(m, metas)
  title = filter(x -> get(x, "propertyUri", "") == "http://nakala.fr/terms#title", metas)[1]

  item = Dict(
    get(title, "value", "noTitle") => identifier
  )
  push!(list, item)
end

list

### Chercher une donnée précise

In [ ]:
title = "Z1J432"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute

status = [ ["pending"], ["published"] ]

datas = get(getUserDatas(scope[1], status[2], title), "data", "nothing")
datas[1] # il peut y avoir plusieurs résultats

### Télécharger les images d'une donnée

In [ ]:
title = "Z1J1149"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(getUserDatas(scope[1], status[2], title), "data", "nothing")
data = datas[1]

downloadFiles(data, title)

### Chercher un fichier précis dans une donnée

In [ ]:
title = "Z1J432"
filenames = ["Z1J432_0004.JPG", "Z1J432_0005.JPG", "Z1J432_0006.JPG"]

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
status = [ ["pending"], ["published"] ]

datas = get(getUserDatas(scope[1], status[2], title), "data", "nothing")
data = datas[1]

getFilesFromData(data, filenames)

## Données

### Publier une donnée avec des fichiers

In [ ]:
include("functions.jl")
path = joinpath(@__DIR__, "datasToSubmit")
directories = listFile(path)
#directories # importée depuis listFile.jl
[postData(directory) for directory in directories]

### Ajouter des fichiers à une données

In [ ]:
include("functions.jl")
dataIdentifier = "10.34847/nkl.765cs153"
path = joinpath(@__DIR__, "test")
postedfiles = [postFile(joinpath(path, file)) for file in readdir(path)]

[addFileToData(dataIdentifier, file) for file in postedfiles]